In [2]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
load_dotenv()

True

In [3]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{num}의 10배는?")
prompt

PromptTemplate(input_variables=['num'], template='{num}의 10배는?')

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")
chain = prompt | model

In [7]:
chain.invoke({"num": 5})

AIMessage(content='50', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a4e831a2-ff33-4088-b73c-3d8fe1967626-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16})

### RunnablePassthrough

In [10]:
from langchain_core.runnables import RunnablePassthrough

In [12]:
runable_chain = {"num": RunnablePassthrough()} | prompt | model
runable_chain.invoke(10)

AIMessage(content='100', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7a8b3b68-fe9e-4262-ba4e-0d13810069a8-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16})

### RunnablePassthrough.assign

In [13]:
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 3})

{'num': 3, 'new_num': 9}

### RunnableParallel

In [14]:
from langchain_core.runnables import RunnableParallel

# 여러 Runnable instance를 병렬로 실행
runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    extra = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified = lambda x: x["num"] + 1
)

runnable.invoke({"num": 2})

{'passed': {'num': 2}, 'extra': {'num': 2, 'mult': 6}, 'modified': 3}

In [15]:
chain1 = ({"num": RunnablePassthrough()}
        | PromptTemplate.from_template("{num}의 10배는?")
        | model
)
chain2 = ({"num": RunnablePassthrough()}
        | PromptTemplate.from_template("{num}의 1/10배는?")
        | model
)

combined_chain = RunnableParallel(a=chain1, b=chain2)
combined_chain.invoke({"num":10})

{'a': AIMessage(content='100', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 20, 'total_tokens': 21}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-53ffa7cf-b1fd-45af-ae0a-52b8521eb86b-0', usage_metadata={'input_tokens': 20, 'output_tokens': 1, 'total_tokens': 21}),
 'b': AIMessage(content='1입니다. 1을 10으로 나누면 0.1이 되고, 소수점 이하를 버리면 1이 됩니다.', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 22, 'total_tokens': 61}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b895e6b2-6465-48c8-83aa-25c19e5bc66b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 39, 'total_tokens': 61})}

### RunnableLambda

In [16]:
from langchain_core.runnables import RunnableLambda

# runnable.invoke = {'passed': {'num': 2}, 'extra': {'num': 2, 'mult': 6}, 'modified': 3}

def extra(x):
    ext = x["extra"]
    return int(ext["num"]) * int(ext["mult"])

(runnable | RunnableLambda(extra)).invoke({"num": 3})


27